# Import Library

In [5]:
# Load, explore and plot data
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
# %matplotlib inline
# Train test split
from sklearn.model_selection import train_test_split
# Text pre-processing
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# Modeling
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.semi_supervised import LabelSpreading
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

import joblib


In [11]:
%run notebook_core_utils.ipynb
%run notebook_text_processor.ipynb
%run notebook_core_mappers.ipynb

# Contacts (Common)

In [6]:
# Variables

# factor_column_name = 'finding_description'

# factors_columns = [ factor_column_name ]

# narrative_columns = ['narrative_01', 'narrative_02']

# filtered_columns = ['event_id'] + factors_columns + narrative_columns

# Functions (Common)

In [7]:
# #----------- Data Manipulation Func ----------
# def get_sample_size(df, percentage):
#     row_count = df.shape[0]
#     return round(row_count * (percentage / 100))

# def get_data(ds_name):

#     path_prefix = '../data/local_ex/astapm'

#     asrs = pd.read_csv(f'{path_prefix}/{ds_name}/{ds_name}.csv', low_memory=False)
#     asrs_narrative = pd.read_csv(f'{path_prefix}/{ds_name}/{ds_name}_narrative.csv', low_memory=False)
#     asrs_factors = pd.read_csv(f'{path_prefix}/{ds_name}/{ds_name}_safety_factors.csv', low_memory=False)

#     data_df = pd.merge(asrs, asrs_narrative, on='event_id', how='left')
#     data_df = pd.merge(data_df, asrs_factors, on='event_id', how='left')

#     # data_df = data_df[data_df['year'] == 2023]
#     # data_df = data_df[(data_df['year'] >= 2008) & (data_df['year'] <= 2023)]

#     df = data_df[filtered_columns].copy()
#     return df

# # Function to vectorize the 'Combined_Factors' column
# def get_X_vec(df, col_name):
#     X = df[col_name]
#     vectorizer = TfidfVectorizer()
#     X_vec = vectorizer.fit_transform(X)
#     return X_vec, vectorizer

# def get_Y_encoder(df, label):
#     labels = df[label].values
#     label_encoder = LabelEncoder()
#     Y = label_encoder.fit_transform(labels)
#     print('Shape of label tensor:', Y.shape)
#     return label_encoder, Y

# #----------- Model Train/Predict Func---------
# def train_LS(df, ds_name='asrs'):
#     # Assuming 'data' contains 'Combined_Factors' and 'HFACS_Category_Value'
#     ls_df = df.copy()

#     # Step 1: Prepare the data (drop rows with missing values)
#     ls_df = ls_df.dropna(subset=['HFACS_Category_Value', factor_column_name])

#     # Encode the 'HFACS_Category_Value'
#     label_encoder, y = get_Y_encoder(ls_df, 'HFACS_Category_Value')
#     unmapped_encoder = label_encoder.transform(['Unmapped'])[0]
    
#     # Set unlabeled data points to -1 (required by LabelSpreading)
#     y[y == unmapped_encoder] = -1

#     # Step 2: Vectorize the 'Combined_Factors' column using TF-IDF
#     X_vec, vectorizer = get_X_vec(ls_df, factor_column_name)

#     # Step 3: Split the labeled data into train and test sets (for evaluation)
#     X_train, X_test, y_train, y_test = train_test_split(X_vec[y != -1], y[y != -1], test_size=0.2, random_state=42)

#     # Step 4: Create and fit the LabelSpreading model using ALL data (labeled + unlabeled)
#     label_prop_model = LabelSpreading()
#     label_prop_model.fit(X_vec, y)

#     # Step 5: Make predictions on the test set (only labeled data for evaluation)
#     y_test_predict = label_prop_model.predict(X_test)
#     accuracy = accuracy_score(y_test, y_test_predict)
#     print("Accuracy on test data:", accuracy)

#     # Optionally show confusion matrix
#     show_confusion_matrix(label_prop_model, X_test, y_test, label_encoder)

#     # Step 6: Predict the labels for the full dataset (including unlabeled data)
#     y_full_predict = label_prop_model.predict(X_vec)

#     # Handle -1 predictions by replacing them with 'Unmapped'
#     y_full_predict = np.where(y_full_predict == -1, unmapped_encoder, y_full_predict)

#     # Decode the predicted labels back to their original categories
#     y_decoded = label_encoder.inverse_transform(y_full_predict)

#     # Add the predicted labels back to the DataFrame
#     ls_df['HFACS_Category_Value_Predict'] = y_decoded

#     save_model_LS(label_prop_model, vectorizer, label_encoder, unmapped_encoder, ds_name)

#     return label_prop_model, label_encoder, unmapped_encoder, vectorizer

# def predict_LS(df, ds_name='asrs', sample_size=0):

#     model, vectorizer, label_encoder, unmapped_encoder = load_model_LS(ds_name)

#     # New set of data for prediction
#     if sample_size > 0:
#         labeled_data =  df.tail(sample_size).copy()
#     else:
#         labeled_data =  df

#     # Drop rows with NaN in 'Combined_Factors'
#     labeled_data = labeled_data.dropna(subset=[factor_column_name])

#     # Vectorize the new set of data
#     # X_vec = get_LS_X(labeled_data)
#     X_vec = vectorizer.transform(labeled_data[factor_column_name])  

#     # Predict using the trained LabelSpreading model
#     y = model.predict(X_vec)

#     # Replace -1 values with 'Unmapped' placeholder
#     y = np.where(y == -1, unmapped_encoder, y)

#     # Decode the predicted labels
#     y_decoded = label_encoder.inverse_transform(y)

#     # Add predictions to the new DataFrame
#     labeled_data['HFACS_Category_Value_Predict'] = y_decoded
#     return labeled_data
# #----------- Model Train/Predict Func---------

# #----------- Visualization Func ----------
# # Show Label Distribution
# def show_label(df, label_col):
#     plt.figure(figsize=(8,6))
#     sns.countplot(df[label_col])
#     plt.title('The distribution of Primary problem')

# # confusion_matrix
# def show_confusion_matrix(model, X_test, y_test, label_encoder):
#     # Predict the classes for the test set
#     y_pred = model.predict(X_test)

#     # Generate the confusion matrix
#     conf_matrix = confusion_matrix(y_test, y_pred)

#     # Plot the confusion matrix
#     plt.figure(figsize=(10,8))
#     sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
#     plt.xlabel('Predicted Label')
#     plt.ylabel('True Label')
#     plt.title('Confusion Matrix')
#     plt.show()
# #----------- Visualization Func ----------

# #----------- Model Save/Load Func ----------
# # https://www.analyticsvidhya.com/blog/2023/02/how-to-save-and-load-machine-learning-models-in-python-using-joblib-library/
# def save_model_LS(model, vectorizer, label_encoder, unmapped_encoder, name):
#     # Save both the model and vectorizer to a file
#     dump_name = f'model_LS_{name}.pkl'
#     joblib.dump((model, vectorizer, label_encoder, unmapped_encoder), dump_name)
#     print(f"{name} Model and vectorizer saved successfully")

# def load_model_LS(name):
#     # Load both the model and vectorizer from the file
#     dump_name = f'model_LS_{name}.pkl'
#     model, vectorizer,label_encoder, unmapped_encoder = joblib.load(dump_name)
#     print("Model and vectorizer loaded successfully")
#     return model, vectorizer, label_encoder, unmapped_encoder
# #----------- Model Save/Load Func ----------

# #------------- HFACS Mapping ----------
# HFACS_DICTIONARY = {
#     # Unmapped
#     -1: ('Unmapped', 'Unmapped', 'Unmapped', 'Unmapped'),
#     # Unsafe Acts
#     111: ('Level 1', 'Unsafe acts', 'Errors', 'Decision/Skill-based/Perceptual Errors'),
#     # 112: ('Level 1', 'Unsafe acts', 'Errors', 'Skill-based Errors'),
#     # 113: ('Level 1', 'Unsafe acts', 'Errors', 'Perceptual Errors'),
#     121: ('Level 1', 'Unsafe acts', 'Violations', 'Routine / Exceptional Violations'),
#     # 122: ('Level 1', 'Unsafe acts', 'Violations', 'Exceptional Violations'),

#     # Preconditions for Unsafe Acts
#     211: ('Level 2', 'Preconditions for Unsafe Acts', 'Environmental Factors', 'Physical Environment'),
#     212: ('Level 2', 'Preconditions for Unsafe Acts', 'Environmental Factors', 'Technological Environment'),
#     221: ('Level 2', 'Preconditions for Unsafe Acts', 'Conditions of Operators', 'Adverse Mental State/Adverse Physiological State/Physical Limitations'),
#     # 222: ('Level 2', 'Preconditions for Unsafe Acts', 'Conditions of Operators', 'Adverse Physiological State'),
#     # 223: ('Level 2', 'Preconditions for Unsafe Acts', 'Conditions of Operators', 'Physical Limitations'),
#     231: ('Level 2', 'Preconditions for Unsafe Acts', 'Personnel Factors', 'Crew Resource Management/Personal Readiness'),
#     # 232: ('Level 2', 'Preconditions for Unsafe Acts', 'Personnel Factors', 'Personal Readiness'),
    
#     # Unsafe Supervision
#     321: ('Level 3', 'Unsafe Supervision', 'Unsafe Supervision', 'Planned Inappropriate Operations'),
#     311: ('Level 3', 'Unsafe Supervision', 'Unsafe Supervision', 'Inadequate Supervision'),
#     331: ('Level 3', 'Unsafe Supervision', 'Unsafe Supervision', 'Failure to Correct Known Problems'),
#     341: ('Level 3', 'Unsafe Supervision', 'Unsafe Supervision', 'Supervisory Violations'),
    
#     # Organizational Influences
#     411: ('Level 4', 'Organizational Influences', 'Organizational Influences', 'Organizational Climate/Organizational Process/Resource Management'),
#     # 421: ('Level 4', 'Organizational Influences', 'Organizational Influences', 'Organizational Process'),
#     # 431: ('Level 4', 'Organizational Influences', 'Organizational Influences', 'Resource Management')
# }
# #------------- HFACS Mapping ----------

# ASRS

## Function | Variables | Constants (ASRS) 

In [12]:


# #------------- ASRS Manual Mapping ----------
# keyword_pf_mappings_asrs = {
#     "Aircraft": 212,                                        # Preconditions for Unsafe Acts => Technological Environment
#     "Airport": 211,                                         # Preconditions for Unsafe Acts => Physical Environment
#     "Airspace Structure": 212,                              # Preconditions for Unsafe Acts => Technological Environment
#     "ATC Equipment / Nav Facility / Buildings": 212,        # Preconditions for Unsafe Acts => Technological Environment
#     "Chart Or Publication": 411, #421,                            # Organizational Influences => Organizational Process
#     "Company Policy": 411,                                  # Organizational Influences => Organizational Climate
#     "Environment - Non Weather Related": 211,               # Preconditions for Unsafe Acts => Physical Environment
#     "Equipment / Tooling": 212,                             # Preconditions for Unsafe Acts => Technological Environment
#     "Human Factors": 11,                                    # Unsafe Acts => Errors or Violations
#     "Incorrect / Not Installed / Unavailable Part": 411, #431,    # Organizational Influences => Resource Management
#     "Logbook Entry": 411, #421,                                   # Organizational Influences => Organizational Process
#     "Manuals": 411, #421,                                         # Organizational Influences => Organizational Process
#     "MEL": 411, #421,                                             # Organizational Influences => Organizational Process
#     "Procedure": 411, #421,                                       # Organizational Influences => Organizational Process
#     "Software and Automation": 212,                         # Preconditions for Unsafe Acts => Technological Environment
#     "Staffing": 411, #431,                                        # Organizational Influences => Resource Management
#     "Weather": 211                                          # Preconditions for Unsafe Acts => Physical Environment
# }

# # keyword_pf_balance_mappings_asrs = {
# #     "Aircraft": 212,                                        # Preconditions for Unsafe Acts => Technological Environment
# #     "Airport": 211,                                         # Preconditions for Unsafe Acts => Physical Environment
# #     "Airspace Structure": 212,                              # Preconditions for Unsafe Acts => Technological Environment
# #     "ATC Equipment / Nav Facility / Buildings": 212,        # Preconditions for Unsafe Acts => Technological Environment
# #     "Chart Or Publication": 421,                            # Organizational Influences => Organizational Process
# #     "Company Policy": 411,                                  # Organizational Influences => Organizational Climate
# #     "Environment - Non Weather Related": 211,               # Preconditions for Unsafe Acts => Physical Environment
# #     "Equipment / Tooling": 212,                             # Preconditions for Unsafe Acts => Technological Environment
# #     "Human Factors": 11,                                    # Unsafe Acts => Errors or Violations
# #     "Incorrect / Not Installed / Unavailable Part": 431,    # Organizational Influences => Resource Management
# #     "Logbook Entry": 421,                                   # Organizational Influences => Organizational Process
# #     "Manuals": 421,                                         # Organizational Influences => Organizational Process
# #     "MEL": 421,                                             # Organizational Influences => Organizational Process
# #     "Procedure": 421,                                       # Organizational Influences => Organizational Process
# #     "Software and Automation": 212,                         # Preconditions for Unsafe Acts => Technological Environment
# #     "Staffing": 431,                                        # Organizational Influences => Resource Management
# #     "Weather": 211                                          # Preconditions for Unsafe Acts => Physical Environment
# # }

# keyword_hf_mappings_asrs = {
#     "Communication Breakdown": 231,     # Preconditions for Unsafe Acts => Personnel Factors => Crew Resource Management)
#     "Confusion": 111, #113,                   # Perceptual Errors
#     "Distraction": 111, #112,                 # Skill-based Errors
#     "Fatigue": 221,                     # Adverse Mental State
#     "Human-Machine Interface": 212,     # Technological Environment
#     # "Other / Unknown": (-1, -1, -1),  # General Error (Unspecified)
#     "Physiological - Other": 221, #222,       # Adverse Physiological State
#     "Situational Awareness": 221,       # Conditions of Operators → Adverse Mental State
#     "Time Pressure": 221,               # Adverse Mental State
#     "Training / Qualification": 111, #112,    # Personal Readiness
#     "Troubleshooting": 111, #112,             # Skill-based Errors
#     "Workload": 221                     # Adverse Mental State
# }


#------------- ASRS Manual Mapping ----------

#------------- Manual Labeling Func ----------

# keyword_hf_mappings_asrs = AUTO_LABELING_DICTIONARY["asrs"]["HF_DICTIONARY"]
# keyword_pf_mappings_asrs = AUTO_LABELING_DICTIONARY["asrs"]["PF_DICTIONARY"]

# HFACS_DICTIONARY = HFACS_DICTIONARY["hfacs"]
# factor_column_name = CoreUtils.get_constant()["FACTOR_COL_NAME"]

# def map_hf_asrs(factors):
#     # Convert to lowercase for consistent keyword matching
#     factors = str(factors).lower()

#     split_factors = factors.split(':')
#     if(len(split_factors) == 1):
#         # Only include Human factor, map with # Skill-based Errors TODO Need to improve
#         return 111 #112
    
#     for keyword, mapping in keyword_hf_mappings_asrs.items():
#         # print(keyword.lower(), '=>>>', factors)
#         if keyword.lower() in factors:  # Check if keyword is in the combined issues string
#             # print(keyword.lower(), '=>>>', factors)
#             return mapping
#     return -1

# # Function to map combined issues to HFACS based on keyword mappings
# def map_hfacs_asrs(factors):
#      # Convert to lowercase for consistent keyword matching
#     factors = str(factors).lower()

#     for keyword, mapping in keyword_pf_mappings_asrs.items():
#         if keyword.lower() in factors:  # Check if keyword is in the combined issues string
#             # print(keyword.lower(), '=>>>', factors)
#             if(mapping == 11):
#                 # print(mapping)
#                 return map_hf_asrs(factors)
#             return mapping
#     return -1  # Default to unmapped if no match found

# def manual_labeling_asrs(df, sample_size=0):

#     # df_tail = df.tail(sample_size).copy()

#     if sample_size > 0:
#         df_tail =  df.sample(n=sample_size, random_state=42)
#     else:
#         df_tail = df
        
#     print('sample_size=', df_tail.shape)
    
#     df_tail.replace('', pd.NA, inplace=True)
#     # print_column = ['ACN'] + factors_column

#     # df_tail[print_column].value_counts()
#     # df_tail.isnull().sum()

#     # Load the dataset (update the file path as needed)
#     data = df_tail.copy()
#     # Apply the function to each 'Combined_Issues' row and map to HFACS
#     data['HFACS_Category_Index'] = data[factor_column_name].apply(map_hfacs_asrs)

#     # Split the 'HFACS_Mapped' into two new columns: 'HFACS_Category' and 'HFACS_Level'
#     data['HFACS_Category_Value'] = data['HFACS_Category_Index'].apply(lambda x: HFACS_DICTIONARY[x][3])
#     data['HFACS_Level'] = data['HFACS_Category_Index'].apply(lambda x: HFACS_DICTIONARY[x][0])

#     # # Summarize the mapped HFACS categories and levels
#     hfacs_summary = data.groupby(['HFACS_Category_Value']).size().reset_index(name='Count')
#     print(hfacs_summary)
#     return data

# #------------- Manual Labeling Func ----------


# NTSB

## Function | Variables | Constants (NTSB) 

In [9]:
#------------- ASRS Manual Mapping ----------
# keyword_mappings_ntsb = {
#     # Unsafe Acts => Skill-based Errors
#     "Personnel issues-Task performance": 112,
#     # Unsafe Acts => Decision Errors
#     "Personnel issues-Action/decision": 111,
#      # Unsafe Acts => Perceptual Errors
#     "Personnel issues-Psychological-Perception/orientation/illusion": 113,
#     # Unsafe Acts => Violations
#     "Personnel issues-Miscellaneous-Intentional act": 121,

#     # Preconditions for Unsafe Acts => Physical Environment
#     'Environmental issues-Physical environment': 211,
#     'Environmental issues-Conditions/weather/phenomena-Turbulence': 211,
#     'Environmental issues-Task environment': 211,

#     # Preconditions for Unsafe Acts => Technological Environment
#     'Environmental issues-Operating environment': 212,
#     'Aircraft-Aircraft systems': 212,

#     # Preconditions for Unsafe Acts => Adverse Mental State
#     'Personnel issues-Physical-Alertness/Fatigue': 221,
#     'Personnel issues-Psychological-Attention/monitoring': 221,
#     'Personnel issues-Psychological-Personality/attitude': 221,
#     'Personnel issues-Psychological-Mental/emotional state': 221,

#     # Preconditions for Unsafe Acts => Adverse Physiological State
#     'Personnel issues-Psychological': 222,
#     'Personnel issues-Physical-Impairment/incapacitation': 222,
#     'Personnel issues-Physical-Health/Fitness': 222,

#     # Preconditions for Unsafe Acts => Physical Limitations
#     'Personnel issues-Physical-Sensory ability/limitation': 223,

#     # Preconditions for Unsafe Acts => Crew Resource Management
#     'Lack of communication': 231,

#     # Preconditions for Unsafe Acts => Personal Readiness
#     "Personnel issues-Experience/knowledge": 232,

#     # Organizational Influences => Organizational Climate
#     'Organizational issues-Management-Culture': 411,
#     'Organizational issues-Management-Communication (organizational)': 411,

#     # Organizational Influences =>  Organizational Process
#     'Organizational issues-Management-Scheduling':  421,
#     'Organizational issues-Management-Policy/procedure':  421,
#     'Organizational issues-Management-(general)-(general)-Operator':  421,
#     'Organizational issues-Support/oversight/monitoring': 421,

#     # Organizational Influences =>  Resource Management
#     'Organizational issues-Management-Resources': 431,
#     'Organizational issues-Development-Selection/certification/testing': 431,
# }
# #------------- ASRS Manual Mapping ----------

# keyword_mappings_ntsb = AUTO_LABELING_DICTIONARY["ntsb"]

# # Function to map combined issues to HFACS based on keyword mappings
# def map_hfacs_ntsb(factors):

#      # Convert to lowercase for consistent keyword matching
#     factors = str(factors).lower()

#     for keyword, mapping in keyword_mappings_ntsb.items():
#         # Check if keyword is in the factors string
#         if keyword.lower() in factors:
#             # print(keyword.lower(), '=>>>', factors)
#             return mapping
#      # Default to unmapped if no match found
#     return -1

# def manual_labeling_ntsb(df, sample_size):

#     # df_tail = df.tail(sample_size).copy()

#     df_tail = df.sample(n=sample_size, random_state=42)
    
#     df_tail.replace('', pd.NA, inplace=True)
#     # print_column = ['ACN'] + factors_column

#     # df_tail[print_column].value_counts()
#     # df_tail.isnull().sum()

#     # Load the dataset (update the file path as needed)
#     data = df_tail.copy()
#     # Apply the function to each 'Combined_Issues' row and map to HFACS
#     data['HFACS_Category_Index'] = data[factor_column_name].apply(map_hfacs_ntsb)

#     # Split the 'HFACS_Mapped' into two new columns: 'HFACS_Category' and 'HFACS_Level'
#     data['HFACS_Category_Value'] = data['HFACS_Category_Index'].apply(lambda x: HFACS_DICTIONARY[x][3])
#     data['HFACS_Level'] = data['HFACS_Category_Index'].apply(lambda x: HFACS_DICTIONARY[x][0])

#     # # Summarize the mapped HFACS categories and levels
#     hfacs_summary = data.groupby(['HFACS_Category_Value']).size().reset_index(name='Count')
#     print(hfacs_summary)
#     return data

# def train_LS_NTSB(df, factor_label_name='finding_description'):
#     # Assuming 'data' contains 'Combined_Factors' and 'HFACS_Category_Value'
#     ls_df = df.copy()

#     # Step 1: Prepare the data (drop rows with missing values)
#     ls_df = ls_df.dropna(subset=['HFACS_Category_Value', factor_label_name])

#     # Encode the 'HFACS_Category_Value'
#     label_encoder, y = get_Y_encoder(ls_df, 'HFACS_Category_Value')
#     unmapped_encoder = label_encoder.transform(['Unmapped'])[0]
    
#     # Set unlabeled data points to -1 (required by LabelSpreading)
#     y[y == unmapped_encoder] = -1

#     # Step 2: Vectorize the 'Combined_Factors' column using TF-IDF
#     X_vec, vectorizer = get_X_vec(ls_df, factor_label_name)

#     # Step 3: Split the labeled data into train and test sets (for evaluation)
#     X_train, X_test, y_train, y_test = train_test_split(X_vec[y != -1], y[y != -1], test_size=0.2, random_state=42)

#     # Step 4: Create and fit the LabelSpreading model using ALL data (labeled + unlabeled)
#     label_prop_model = LabelSpreading()
#     label_prop_model.fit(X_vec, y)

#     # Step 5: Make predictions on the test set (only labeled data for evaluation)
#     y_test_predict = label_prop_model.predict(X_test)
#     accuracy = accuracy_score(y_test, y_test_predict)
#     print("Accuracy on test data:", accuracy)

#     # Optionally show confusion matrix
#     show_confusion_matrix(label_prop_model, X_test, y_test, label_encoder)

#     # Step 6: Predict the labels for the full dataset (including unlabeled data)
#     y_full_predict = label_prop_model.predict(X_vec)

#     # Handle -1 predictions by replacing them with 'Unmapped'
#     y_full_predict = np.where(y_full_predict == -1, unmapped_encoder, y_full_predict)

#     # Decode the predicted labels back to their original categories
#     y_decoded = label_encoder.inverse_transform(y_full_predict)

#     # Add the predicted labels back to the DataFrame
#     ls_df['HFACS_Category_Value_Predict'] = y_decoded

#     return label_prop_model, label_encoder, unmapped_encoder, vectorizer

# def predict_LS_NTSB(df, sample_size=0):

#     model, vectorizer, label_encoder, unmapped_encoder = load_model_LS('ntsb')

#     # New set of data for prediction
#     if sample_size > 0:
#         labeled_data =  df.tail(sample_size).copy()
#     else:
#         labeled_data =  df

#     # Drop rows with NaN in 'Combined_Factors'
#     labeled_data = labeled_data.dropna(subset=['finding_description'])

#     # Vectorize the new set of data
#     # X_vec = get_LS_X(labeled_data)
#     X_vec = vectorizer.transform(labeled_data['finding_description'])  

#     # Predict using the trained LabelSpreading model
#     y = model.predict(X_vec)

#     # Replace -1 values with 'Unmapped' placeholder
#     y = np.where(y == -1, unmapped_encoder, y)

#     # Decode the predicted labels
#     y_decoded = label_encoder.inverse_transform(y)

#     # Add predictions to the new DataFrame
#     labeled_data['HFACS_Category_Value_Predict'] = y_decoded
#     return labeled_data
